In [30]:
from helper import *
from helper2 import *

In [33]:
import os
notebook_path = os.path.abspath('')
print(notebook_path)

/home/dimitris/Documents/github/EPF_ABA23


In [25]:
# Let's see if we can run Short term.ipynb here
from notebook import notebookapp
import requests
import json

servers = list(notebookapp.list_running_servers())
print(servers)

[]


In [26]:
from IPython.display import IFrame
IFrame('Short term.ipynb', width='100%', height='500px')

In [27]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from operator import itemgetter

In [28]:
import statsmodels.api as sm

In [32]:
# this is to call the linear regression importance feature
lr_imp = lin_reg_feat_importance(x_train, y_train, x_test, y_test)

NameError: name 'lin_reg_feat_importance' is not defined

In [3]:
# plot
var_imp_plot(lr_imp, 'Linear regression feature importance')

NameError: name 'lr_imp' is not defined

In [4]:
# vlepoume
# def feature_random_forest(x_train, y_train):
#     rf_selector = RandomForestClassifier(n_estimators=100)
#     fit = rf_selector.fit(x_train, y_train)

#     cols = pd.DataFrame(x_train.columns)
#     ranking = pd.DataFrame(fit.feature_importances_)
#     rfe_selector = pd.concat([cols,ranking], axis=1)
#     rfe_selector.columns = ['Feature','Ranking']  # name the dataframe columns

#     rfe_selector = rfe_selector.nlargest(41,'Ranking').reset_index().drop('index',axis=1)
#     print("Num Features: %s" % (fit.n_features_))
#     print(rfe_selector)